https://zhuanlan.zhihu.com/p/70355773

In [1]:
import numpy as np
import torch
from IPython.display import display, Image
import os
from os import listdir
import re
from tqdm import tqdm
import pandas as pd
device = ('cuda' if torch.cuda.is_available() else 'cpu')
import sys

In [2]:
torch.cuda.is_available()

False

# POS Tagging

# 首先有一句话已经标准好了词性

display(Image("/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP/图片/C7A5E18D-1EA1-413F-BCF8-5A650092435D.png"))

def display_img(img_path):
    return display(Image("{}{}{}".format(os.getcwd(),"/I5_algorithm/图片/",img_path)))
display_img("C7A5E18D-1EA1-413F-BCF8-5A650092435D.png")

## 加载词性标注数据

In [3]:
#dirpath = "/media/manman/文档/算法实验/I5_algorithm/NLP数据集合/词性标注数据集/国家语委人工词性标注语料"

In [4]:
def list_file_path(dirpath):
    return [os.path.join(dirpath,dir) for dir in listdir(dirpath)]

def replace_lambda(strings,symbles=[' ','\ufeff'],Replace_the_symbol="\n"):
    srcrep = {i:Replace_the_symbol for i in symbles }
    rep = dict((re.escape(k), v) for k, v in srcrep.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], strings)

def clean_page(filepath):
    file = open(filepath,'r').readlines()
    sentence_tagging = ''.join([replace_lambda('{}{}'.format('sentence/sentence',sentence)) for sentence in file])
    split_pos_tagging = re.split(r'\n',sentence_tagging)
    remove_empty = list(map(lambda x : x.split('/') ,list(filter(lambda x : x!='' , split_pos_tagging))))
    return np.array([['page','page']]+remove_empty) 

def load_pos_tags(dirpath):
    init_pos_tags = []
    cl = []
    for dir in tqdm(listdir(dirpath),desc='加载词性标注文件'):
        one_page = clean_page(os.path.join(dirpath,dir))
        for pos in one_page:
            if (len(pos) == 2) & ('' not in pos):
                init_pos_tags.append(pos)
            else:
                cl.append(pos)
    return np.array(init_pos_tags),cl

In [15]:
pos_tag_path = os.getcwd().replace('NlP/','')+"数据集合/词性标注数据集/国家语委人工词性标注语料"
pos_tags_ , cl = load_pos_tags(pos_tag_path)
#pos_tag_12 ,cl_12 = load_pos_tags(pos_tag_path)

加载词性标注文件: 100%|██████████| 1978/1978 [00:23<00:00, 84.94it/s] 


In [16]:
tags,words = np.unique(pos_tags_[:,1]), np.unique(pos_tags_[:,0])
pos_tags_

array([['page', 'page'],
       ['sentence', 'sentence'],
       ['两', 'm'],
       ...,
       ['一样', 'a'],
       ['。', 'w'],
       ['sentence', 'sentence']], dtype='<U25')

In [17]:
def check_prob(word,tage,m,n,pos_tags):
    check_same = (pos_tags==np.array([word,tage]))
    return (check_same.dot(np.ones(2))==2).dot(np.ones(m))/m

In [18]:
mfull,nfull = pos_tags_.shape
%time check_prob('两', 'm',mfull,nfull,pos_tags_)

CPU times: user 199 ms, sys: 43.6 ms, total: 242 ms
Wall time: 160 ms


0.001712517681875372

def stats_transfer_matrix(pos_tags):
    unique_words,unique_tags = np.unique(pos_tags[:,0]),np.unique(pos_tags[:,1])
    m,n = unique_words.size,unique_tags.size
    mfull,nfull = pos_tags.shape
    init_arr = np.zeros((m,n))#.astype(str)
    tempm , tempn = -1,-1
    for word in tqdm(unique_words,desc='计算词性频率'):
        tempm += 1
        for tag in unique_tags:
            tempn += 1
            if np.array([word,tag]) in pos_tags:
                init_arr[tempm,tempn] += check_prob(word,tag,mfull,nfull,pos_tags)#str(word)+str(tag)#
        tempn = -1
    to_table = pd.DataFrame(init_arr.T,columns=unique_words,index =unique_tags)
    to_table.to_csv("/media/manman/文档/算法实验/I5_algorithm/NLP数据集合/词性标注数据集/词性标注概率转移矩阵/词性标注概率转移矩阵.csv")
    return {"table":to_table,"arr":init_arr}

stats_transfer_matrix(pos_tags_)['table']

In [19]:
def filter_pos_tag(pos_tags):
    filter_index = np.array([(pos_tags[:,1] != 'page')*(pos_tags[:,1] != 'sentence')])[0]
    clear_pos_tags = pos_tags[filter_index]
    return clear_pos_tags

In [20]:
clear_pos_tags = filter_pos_tag(pos_tags_)

In [24]:
def is_GPU(tensor):
    '''
    函数名 : cpu 或者 gpu 自动推断函数
    :param tensor: tensor结构数据
    :return: 根据系统转cpu或者gpu结构
    '''
    if torch.cuda.is_available():
        return tensor.cuda()
    return tensor

def str_coder(pos_tags):
    countm ,countn = pos_tags.shape
    unique_words,counter_words= np.unique(pos_tags[:,0],return_counts=True)
    unique_tags,counter_tags = np.unique(pos_tags[:,1],return_counts=True)
    words_dict = dict(zip(unique_words,np.arange(unique_words.size)))
    tags_dict = dict(zip(unique_tags,np.arange(unique_tags.size)))
    strings_to_code = torch.tensor([[words_dict[pos_tag[0]],tags_dict[pos_tag[1]]] for pos_tag in tqdm(pos_tags,desc='词转编码')])
    return strings_to_code

def check_prob_from_tensor(pos_tag_code,m,strings_to_code):
    check_same = strings_to_code==pos_tag_code
    return (check_same.sum(axis=1)==2).sum().cpu().numpy()/m

def stats_transfer_matrix_from_tensor(pos_tags):
    strings_to_code = str_coder(pos_tags).cuda()
    strwords,strtags = np.unique(pos_tags[:,0]),np.unique(pos_tags[:,1])
    unique_words,unique_tags = torch.unique(strings_to_code[:,0]).cuda(),torch.unique(strings_to_code[:,1]).cuda()
    m,n = unique_words.size()[0],unique_tags.size()[0]
    #mfull,nfull = strings_to_code.shape
    init_arr = np.zeros((int(m),int(n)))
    tempm , tempn = -1,-1
    for word in tqdm(unique_words,desc='计算词性频率'):
        tempm += 1
        for tag in unique_tags:
            tempn += 1
            temp_pos_tag = torch.tensor([word,tag]).cuda()
            temp_tag_m = (strings_to_code[:,1]==tag).sum().cpu().numpy()
            if temp_pos_tag in strings_to_code:
                '''与整体比较概率'''
                temp_prob = check_prob_from_tensor(temp_pos_tag,temp_tag_m,strings_to_code) 
                init_arr[tempm,tempn] += temp_prob 
        tempn = -1
    to_table = pd.DataFrame(init_arr.T,columns = strwords,index = strtags)
    to_table.to_csv(os.getcwd().replace('NlP','')+"数据集合/词性标注数据集/词性标注概率转移矩阵/词性标注概率转移矩阵.csv")
    return {"table":to_table,"arr":init_arr.T,'tags':strtags,'words':strwords}

In [71]:
def str_coder(pos_tags):
    countm ,countn = pos_tags.shape
    unique_words,counter_words= np.unique(pos_tags[:,0],return_counts=True)
    unique_tags,counter_tags = np.unique(pos_tags[:,1],return_counts=True)
    words_dict = dict(zip(unique_words,np.arange(unique_words.size)))
    tags_dict = dict(zip(unique_tags,np.arange(unique_tags.size)))
    strings_to_code = torch.tensor([[words_dict[pos_tag[0]],tags_dict[pos_tag[1]]] for pos_tag in tqdm(pos_tags,desc='词转编码')])
    return strings_to_code

def check_prob_from_tensor(pos_tag_code,m,strings_to_code):
    check_same = strings_to_code==pos_tag_code
    return (check_same.sum(axis=1)==2).sum()/float(m)

def stats_transfer_matrix_from_tensor(pos_tags):
    strings_to_code = str_coder(pos_tags).cuda()
    strwords,strtags = np.unique(pos_tags[:,0]),np.unique(pos_tags[:,1])
    unique_words,unique_tags = torch.unique(strings_to_code[:,0]).cuda(),torch.unique(strings_to_code[:,1]).cuda()
    m,n = unique_words.size()[0],unique_tags.size()[0]
    init_arr = torch.zeros((int(m),int(n))).cuda()
    tempm , tempn = -1,-1
    for word in tqdm(unique_words,desc='计算词性频率'):
        tempm += 1
        for tag in unique_tags:
            tempn += 1
            temp_pos_tag = torch.tensor([word,tag]).cuda()
            temp_tag_m = (strings_to_code[:,1]==tag).sum()
            if temp_pos_tag in strings_to_code:
                '''与整体比较概率'''
                temp_prob = check_prob_from_tensor(temp_pos_tag,temp_tag_m,strings_to_code) 
                init_arr[tempm,tempn] += temp_prob 
        tempn = -1
    init_arr = init_arr.cpu().numpy()
    to_table = pd.DataFrame(init_arr.T,columns = strwords,index = strtags)
    to_table.to_csv("/media/manman/文档/算法实验/I5_algorithm/NLP数据集合/词性标注数据集/词性标注概率转移矩阵/词性标注概率转移矩阵.csv")
    return {"table":to_table,"arr":init_arr.T,'tags':strtags,'words':strwords}

In [95]:
#transfer_matrix = stats_transfer_matrix_from_tensor(pos_tags_[:100])
#A = transfer_matrix['table']
Atable = pd.read_csv(os.getcwd()+"/词性标注概率转移矩阵.csv")
A = Atable.to_numpy()[1:]
A

array([['c', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['d', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ['e', 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       ['w', 0.004965613549854421, 0.0, ..., 0.0, 0.0,
        8.473743259137238e-05],
       ['ws', 0.0, 0.0004952947003467064, ..., 0.0004952947003467064,
        0.0004952947003467064, 0.0],
       ['x', 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [27]:
transfer_matrix_arr = transfer_matrix["arr"]
transfer_matrix_arr

NameError: name 'transfer_matrix' is not defined

In [28]:
transfer_matrix['tags'].shape

NameError: name 'transfer_matrix' is not defined

In [81]:
def get_Pi(pos_tags):
    tag_arr = pos_tags[:,1]
    unique_tags = np.unique(tag_arr)
    n = 0
    m = unique_tags.size
    arr = np.zeros(m)
    for tag in unique_tags:
        n += 1
        if tag == 'sentence':
            prob = (tag_arr==tag).sum()/m
            arr[n] = prob
            
    #arr = np.array([tags_prob_dict[tag]/init_m for tag in tags_prob_dict])
    return {'arr':arr,'table':pd.DataFrame(arr[None,],columns=unique_tags)}

In [83]:
get_Pi(pos_tags_)['table']

,a,c,d,e,f,h,i,j,k,m,...,tu,tv,u,v,vd,vl,vu,w,ws,x
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
Pi = get_Pi(pos_tags_)['arr']

In [88]:
def bigram_for_postag(pos_tags):    
    tag_arr = pos_tags[:,1]
    unique_tags = np.unique(tag_arr)
    m = unique_tags.size
    combination = np.array([tag_arr[i:i+2] for i in range(m) if i < m-1 ])
    prob = lambda tag : (tag_arr == tag).sum()/m
    table = np.zeros((m,m))
    tm,tb = -1,-1
    for tag1 in tqdm(unique_tags,desc='词性标注概率转移矩阵'):
        tm += 1
        for tag2 in unique_tags:
            tb += 1
            if tag1 != tag2:
                tagrow = np.array([tag1,tag2])
                prob_comb = (combination == tagrow).sum()/m    
                table[tm,tb] = prob_comb
        tb = -1
    return table
B = bigram_for_postag(pos_tags_)

词性标注概率转移矩阵: 100%|██████████| 46/46 [00:00<00:00, 2252.82it/s]


## 模型计算

In [89]:
A.shape ,Pi.shape,B.shape

((46, 92937), (46,), (46, 46))

array([[  0,   2,   4,   6,   8,  10,  12,  14,  16,  18],
       [ 10,  12,  14,  16,  18,  20,  22,  24,  26,  28],
       [ 20,  22,  24,  26,  28,  30,  32,  34,  36,  38],
       [ 30,  32,  34,  36,  38,  40,  42,  44,  46,  48],
       [ 40,  42,  44,  46,  48,  50,  52,  54,  56,  58],
       [ 50,  52,  54,  56,  58,  60,  62,  64,  66,  68],
       [ 60,  62,  64,  66,  68,  70,  72,  74,  76,  78],
       [ 70,  72,  74,  76,  78,  80,  82,  84,  86,  88],
       [ 80,  82,  84,  86,  88,  90,  92,  94,  96,  98],
       [ 90,  92,  94,  96,  98, 100, 102, 104, 106, 108]])

In [27]:
def cut_text(text,lenth):
    textArr = re.findall('.{'+str(lenth)+'}', text) 
    textArr.append(text[(len(textArr)*lenth):]) 
    return textArr

In [28]:
cut_text("hahah",2)

['ha', 'ha', 'h']